In [1]:
import random

import flax
import flax.linen as nn
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax.training.train_state import TrainState
from stable_baselines3.common.buffers import ReplayBuffer

from rlmcmc.env import RLMHEnv
from mcmctoolbox.functoolbox import flat

from tqdm.auto import trange

import toml
from types import SimpleNamespace

import json
import pandas as pd
import bridgestan as bs
from posteriordb import PosteriorDatabase

config = toml.load("./rlmcmc/config/ddpg.toml")
args = SimpleNamespace(**config)

# seeding
random.seed(args.seed)
np.random.seed(args.seed)
key = jax.random.PRNGKey(args.seed)
key, actor_key, qf1_key = jax.random.split(key, 3)

CUDA backend failed to initialize: Found cuPTI version 18, but JAX was built against version 20, which is newer. The copy of cuPTI that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
# Initialize Agent

class QNetwork(nn.Module):
    """
    Critic Network
    """
    @nn.compact
    def __call__(self, x: jnp.ndarray, a: jnp.ndarray):
        x = jnp.concatenate([x, a], -1)
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(1)(x)
        x = nn.softplus(x)
        return x

In [3]:
class Actor(nn.Module):
    action_dim: int

    @nn.compact
    def __call__(self, obs: jnp.ndarray):
        x, mcmc_noise = jnp.split(obs, [self.action_dim-1], axis=1)
        x_star = self.gene_x_star(x, mcmc_noise)

        print("x:", x)
        print("cov_x:", self.low_rank_vect_and_mag(x))
        print("cov_x_star:", self.low_rank_vect_and_mag(x_star))

        log_q_ratio = self.log_q(
            x, x_star, self.cov(x_star)
        ) - self.log_q(
            x_star, x, self.cov(x)
        )

        return jnp.hstack(
            [x_star, jnp.expand_dims(log_q_ratio, axis=1)]
        )

    def low_rank_vect_and_mag(self, x):
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(self.action_dim)(x)
        x = nn.softplus(x)
        return x

    def log_q(self, x, mean, cov):
        return jax.scipy.stats.multivariate_normal.logpdf(
            x, 
            mean=mean,
            cov=cov
        )

    def cov(self, x):
        low_rank_vec, mag = jnp.split(self.low_rank_vect_and_mag(x), [self.action_dim-1], axis=1)
        return jnp.sqrt(
            jnp.einsum(
            "ij,ik->ijk", low_rank_vec, low_rank_vec
            ) + jnp.einsum("ij,jk->ijk", mag, jnp.eye(self.action_dim-1))
        )

    def gene_x_star(self, x, mcmc_noise):
        return x + jnp.einsum(
            "ij, ijk -> ik", mcmc_noise, jnp.sqrt(self.cov(x))
        )

In [4]:
class TrainState(TrainState):
    target_params: flax.core.FrozenDict

In [5]:
# Load DataBase Locally
pdb_path = "/home/congye/Code/PythonProjects/LearningAdaptiveMCMC/Python/posteriordb/posterior_database"
my_pdb = PosteriorDatabase(pdb_path)

# Load Dataset
posterior = my_pdb.posterior("test-AnnulusGaussianMixture-test-AnnulusGaussianMixture")
stan = posterior.model.stan_code_file_path()
data = json.dumps(posterior.data.values())
model = bs.StanModel.from_stan_file(stan, data)

# Gold Standard
gs = posterior.reference_draws()
df = pd.DataFrame(gs)
gs_chains = np.zeros((sum(flat(posterior.information['dimensions'].values())),\
                       posterior.reference_draws_info()['diagnostics']['ndraws']))
for i in range(len(df.keys())):
    s = []
    for j in range(len(df[df.keys()[i]])):
        s += df[df.keys()[i]][j]
    gs_chains[i, :] = s
linv = np.linalg.inv(np.cov(gs_chains))

# Extract log-P-pdf and its gradient
log_p = model.log_density


In [6]:
# Setup env
sample_dim = 2
action_dim = sample_dim + 1
state_dim = sample_dim * 2

total_timesteps=5_000
args.total_timesteps = total_timesteps
args.batch_size = 48

env = RLMHEnv(
    log_target_pdf=log_p,
    sample_dim=sample_dim,
    total_timesteps=total_timesteps
    )
rb = ReplayBuffer(
    args.buffer_size,
    env.observation_space,
    env.action_space,
    device='cpu',
    handle_timeout_termination=False
)

In [7]:
# Start
obs, _ = env.reset()
actor = Actor(action_dim=sample_dim+1)
qf1 = QNetwork()

In [8]:
actor_state = TrainState.create(
    apply_fn=actor.apply,
    params=actor.init(actor_key, obs),
    target_params=actor.init(actor_key, obs),
    tx=optax.adam(learning_rate=args.learning_rate),
)

qf1_state = TrainState.create(
    apply_fn=qf1.apply,
    params=qf1.init(qf1_key, obs, env.action_space.sample()),
    target_params=qf1.init(qf1_key, obs, env.action_space.sample()),
    tx=optax.adam(learning_rate=args.learning_rate),
)

# actor.apply = jax.jit(actor.apply)
# qf1.apply = jax.jit(qf1.apply)

x: [[0. 0.]]
cov_x: [[0.7446178  0.49338648 0.7783866 ]]
cov_x_star: [[1.3485494  0.82532114 0.62665004]]
x: [[0. 0.]]
cov_x: [[0.7446178  0.49338648 0.7783866 ]]
cov_x_star: [[1.3485494  0.82532114 0.62665004]]


In [9]:
@jax.jit
def update_critic(
    actor_state: TrainState,
    qf1_state: TrainState,
    observations: np.ndarray,
    actions: np.ndarray,
    next_observations: np.ndarray,
    rewards: np.ndarray,
    dones: np.ndarray,
):
    # next_state_actions = (actor.apply(actor_state.target_params, next_observations)).clip(-1, 1)
    next_state_actions = (actor.apply(actor_state.target_params, next_observations))
    qf1_next_target = qf1.apply(qf1_state.target_params, next_observations, next_state_actions).reshape(-1)
    next_q_value = (rewards + (1 - dones) * args.gamma * (qf1_next_target)).reshape(-1)

    def mse_loss(params):
        qf1_a_values = qf1.apply(params, observations, actions).squeeze()
        return ((qf1_a_values - next_q_value) ** 2).mean(), qf1_a_values.mean()

    (qf1_loss_value, qf1_a_values), grads = jax.value_and_grad(mse_loss, has_aux=True)(qf1_state.params)
    qf1_state = qf1_state.apply_gradients(grads=grads)
    return qf1_state, qf1_loss_value, qf1_a_values

@jax.jit
def update_actor(
    actor_state: TrainState,
    qf1_state: TrainState,
    observations: np.ndarray,
):
    def actor_loss(params):
        return -qf1.apply(qf1_state.params, observations, actor.apply(params, observations)).mean()

    actor_loss_value, grads = jax.value_and_grad(actor_loss)(actor_state.params)
    actor_state = actor_state.apply_gradients(grads=grads)
    actor_state = actor_state.replace(
        target_params=optax.incremental_update(actor_state.params, actor_state.target_params, args.tau)
    )
    qf1_state = qf1_state.replace(
        target_params=optax.incremental_update(qf1_state.params, qf1_state.target_params, args.tau)
    )
    return actor_state, qf1_state, actor_loss_value

In [10]:
for global_step in trange(args.total_timesteps):
    actions = actor.apply(actor_state.params, obs)
    # actions = (actions + np.random.normal(0, args.exploration_noise)).clip(
    #                 env.action_space.low, env.action_space.high
    #             )
    next_obs, rewards, terminateds, truncateds, infos = env.step(np.array(actions, dtype=np.float64))
    real_next_obs = next_obs.copy()
    rb.add(obs, real_next_obs, actions, rewards, terminateds, infos)

    obs = next_obs

    # Training
    if global_step > args.learning_starts:
        data = rb.sample(args.batch_size)
        qf1_state, qf1_loss_value, qf1_a_values = update_critic(
            actor_state=actor_state,
            qf1_state=qf1_state,
            observations=data.observations.reshape(-1, state_dim).numpy(),
            actions=data.actions.numpy(),
            next_observations=data.next_observations.reshape(-1, state_dim).numpy(),
            rewards=data.rewards.flatten().numpy(),
            dones=data.dones.flatten().numpy()
        )
        if global_step % args.policy_frequency == 0:
            actor_state, qf1_state, actor_loss_value = update_actor(
                actor_state=actor_state,
                qf1_state=qf1_state,
                observations=data.observations.reshape(-1, state_dim).numpy(),
            )

  0%|          | 0/5000 [00:00<?, ?it/s]

x: [[0. 0.]]
cov_x: [[0.7446178  0.49338648 0.7783866 ]]
cov_x_star: [[1.3485494  0.82532114 0.62665004]]
x: [[0.30942744 0.3180327 ]]
cov_x: [[0.7996468  0.48919475 0.75496954]]
cov_x_star: [[1.3359275  0.82440245 0.62776613]]
x: [[0.32139993 0.4286241 ]]
cov_x: [[0.81357074 0.48471713 0.74542654]]
cov_x_star: [[1.2903569 0.9839406 0.6635863]]
x: [[0.8785614  0.79643106]]
cov_x: [[0.90877765 0.482787   0.7129867 ]]
cov_x_star: [[1.5248165 0.6990854 0.66295  ]]
x: [[-0.5026332 -0.5985248]]
cov_x: [[0.66958046 0.49210793 0.8033786 ]]
cov_x_star: [[2.4737372  0.51793146 1.1021866 ]]
x: [[-0.5026332 -0.5985248]]
cov_x: [[0.66958046 0.49210793 0.8033786 ]]
cov_x_star: [[1.5550781  0.72564805 0.6706957 ]]
x: [[-0.39946976 -0.8105141 ]]
cov_x: [[0.669149   0.48089463 0.8070593 ]]
cov_x_star: [[1.6045748  0.6645034  0.70146924]]
x: Traced<ShapedArray(float32[48,2])>with<DynamicJaxprTrace(level=1/0)>
cov_x: Traced<ShapedArray(float32[48,3])>with<DynamicJaxprTrace(level=1/0)>
cov_x_star: Traced

/home/congye/Software/anaconda3/envs/rl/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:263: RuntimeWarning: overflow encountered in cast
  self.rewards[self.pos] = np.array(reward).copy()


x: [[-0.22248662 -2.567095  ]]
cov_x: [[0.6839916  0.31740493 0.82006645]]
cov_x_star: [[2.101118   0.73779595 0.872662  ]]
x: [[-0.22248662 -2.567095  ]]
cov_x: [[0.6839916  0.31740493 0.82006645]]
cov_x_star: [[1.336565   1.8473703  0.80514383]]
x: [[2.8712487  0.41748428]]
cov_x: [[1.1976118  0.59883714 0.8914528 ]]
cov_x_star: [[1.703617  0.9268255 0.6595949]]
x: [[ 0.67784166 -1.9149029 ]]
cov_x: [[0.7777969  0.37022477 0.86023223]]
cov_x_star: [[1.4406897 1.4086577 0.6616742]]
x: [[ 2.2098007  -0.83580256]]
cov_x: [[1.0121156 0.5289325 0.9584023]]
cov_x_star: [[1.3629823  2.053092   0.80077535]]
x: [[ 2.2098007  -0.83580256]]
cov_x: [[1.0121156 0.5289325 0.9584023]]
cov_x_star: [[1.3561847 1.9857546 0.7976027]]
x: [[ 2.2098007  -0.83580256]]
cov_x: [[1.0121156 0.5289325 0.9584023]]
cov_x_star: [[1.6235886  0.98485583 0.63534933]]
x: [[ 0.9642842 -1.6077318]]
cov_x: [[0.81259286 0.4053807  0.8767561 ]]
cov_x_star: [[1.585452  1.0284077 0.628093 ]]
x: [[ 1.1449598 -1.4575828]]
cov_

RuntimeError: log_density() failed with exception: Exception: normal_lpdf: Random variable is -nan, but must be not nan! (in '/home/congye/Code/PythonProjects/LearningAdaptiveMCMC/Python/posteriordb/posterior_database/models/stan/test-AnnulusGaussianMixture.stan', line 30, column 2 to line 33, column 6)


In [11]:
state_list = np.array([i for i in env.store_state]).reshape(-1, state_dim)
# action_list = np.array([i.squeeze() for i in env.store_action])

In [12]:
from matplotlib import pyplot as plt


plt.plot(state_list[:, 0], state_list[:, 1], "o-", alpha=0.2)